# DRL portfolio allocation example

- Based on FinRL documentation
- cleaned up

# Preface

Needed Python packages

- finrl
- gymnasium
- stable-baselines3[extra]

# Parameters

In [1]:
import os
from pathlib import Path

# config = dynamic_import("config", CURRENT_PATH + "/config.py")
# config_tickers = dynamic_import("config_tickers", CURRENT_PATH + "/config_tickers.py")

DATA_FILE = Path("/tmp/yahoo-dow-jones-index.csv")
TRAINED_MODEL_FILE = Path("/tmp/yahoo-dow-jones-index-model.pkl")


DOW_30_TICKER = [
    "AXP",
    "AMGN",
    "AAPL",
    "BA",
    "CAT",
    "CSCO",
    "CVX",
    "GS",
    "HD",
    "HON",
    "IBM",
    "INTC",
    "JNJ",
    "KO",
    "JPM",
    "MCD",
    "MMM",
    "MRK",
    "MSFT",
    "NKE",
    "PG",
    "TRV",
    "UNH",
    "CRM",
    "VZ",
    "V",
    "WBA",
    "WMT",
    "DIS",
    "DOW",
]


# Download and setup trading universe

In [2]:
import pandas as pd

from finrl.marketdata.yahoodownloader import YahooDownloader
from finrl.preprocessing.preprocessors import FeatureEngineer


def setup_data(fname: Path=DATA_FILE) -> pd.DataFrame:
    # Download and save the data in a pandas DataFrame:
    if not fname.exists():
        data_df = YahooDownloader(start_date = '2008-01-01', end_date = '2020-12-01', ticker_list=DOW_30_TICKER).fetch_data()
        data_df.to_csv(fname)
    else:
        data_df = pd.read_csv(fname)

    #  Perform Feature Engineering
    df = FeatureEngineer(
        data_df.copy(),
        use_technical_indicator=True,
        use_turbulence=False
    ).preprocess_data()

    # add covariance matrix as states
    df = df.sort_values(['date','tic'],ignore_index=True)
    df.index = df.date.factorize()[0]

    cov_list = []

    # look back is one year
    lookback=252

    for i in range(lookback,len(df.index.unique())):
        data_lookback = df.loc[i-lookback:i,:]
        price_lookback=data_lookback.pivot_table(index = 'date',columns = 'tic', values = 'close')
        return_lookback = price_lookback.pct_change().dropna()
        covs = return_lookback.cov().values
        cov_list.append(covs)

    df_cov = pd.DataFrame({'date':df.date.unique()[lookback:],'cov_list':cov_list})
    df = df.merge(df_cov, on='date')
    df = df.sort_values(['date','tic']).reset_index(drop=True)
    return df 

df = setup_data()
display(df)


ModuleNotFoundError: No module named 'finrl'

# Train